In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels as sm

### Import csv

In [12]:
filename = './data/ML_IGE_ENTRANTS_MODELE1.csv'

In [13]:
df = pd.read_csv(filename,sep=';',engine='python',index_col='DATEDATA',parse_dates=True)

In [14]:
df.head()

,SITE,CLIENT,FLUX_ACTIVITE,SECTEUR_ACTIVITE,SERVICE_ACTIVITE,MEDIA,TYPEDOSSIER,REGROUPEMENTTDB,TOTAL
DATEDATA,,,,,,,,,
2019-09-18,COURBEVOIE,BCAC,Flux Décès,Prévoyance,PREVOYANCE,Email,Ouverture dossier Décès,Dossiers Prévoyance,4
2019-12-02,COURBEVOIE,BCAC,Flux Prestation,Prestations,PRESTATION,LR/AR,Prestations,Dossiers Prestations,4
2020-01-06,MARSEILLE,ISTYACO GESTION,Flux Noemie,EDI,CONTRAT - COTISATION,Appel,Echanges Noémie,Dossiers EDI,2
2019-08-29,MARSEILLE,ISTYACO GESTION,Flux Devis,Prestations,PRESTATION,Courrier,Devis dentaire,Dossiers Devis,4
2018-08-07,VELIZY,KLESIA PREVOYANCE,Flux Cotisation,Contrats- Cotisations,CONTRAT - COTISATION,Appel,Cotisations individuelles,Dossiers Cotisations,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378795 entries, 0 to 378794
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   SITE              378795 non-null  object
 1   DATEDATA          378795 non-null  object
 2   CLIENT            378787 non-null  object
 3   FLUX_ACTIVITE     378795 non-null  object
 4   SECTEUR_ACTIVITE  378795 non-null  object
 5   SERVICE_ACTIVITE  378795 non-null  object
 6   MEDIA             335016 non-null  object
 7   TYPEDOSSIER       378795 non-null  object
 8   REGROUPEMENTTDB   378795 non-null  object
 9   TOTAL             378795 non-null  int64 
dtypes: int64(1), object(9)
memory usage: 28.9+ MB


### preprocess columns

In [18]:
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [19]:
df.head()

,SITE,CLIENT,FLUX_ACTIVITE,SECTEUR_ACTIVITE,SERVICE_ACTIVITE,MEDIA,TYPEDOSSIER,REGROUPEMENTTDB,TOTAL
DATEDATA,,,,,,,,,
2019-09-18,COURBEVOIE,BCAC,Flux Décès,Prévoyance,PREVOYANCE,Email,Ouverture dossier Décès,Dossiers Prévoyance,4
2019-12-02,COURBEVOIE,BCAC,Flux Prestation,Prestations,PRESTATION,LR/AR,Prestations,Dossiers Prestations,4
2020-01-06,MARSEILLE,ISTYACO GESTION,Flux Noemie,EDI,CONTRAT - COTISATION,Appel,Echanges Noémie,Dossiers EDI,2
2019-08-29,MARSEILLE,ISTYACO GESTION,Flux Devis,Prestations,PRESTATION,Courrier,Devis dentaire,Dossiers Devis,4
2018-08-07,VELIZY,KLESIA PREVOYANCE,Flux Cotisation,Contrats- Cotisations,CONTRAT - COTISATION,Appel,Cotisations individuelles,Dossiers Cotisations,1


### Filter

In [34]:
def filter_df(df, mydict):
    df_filter = df.copy()
    for key,value in mydict.items():
        df_filter = df_filter[df_filter[key]==value]
    return df_filter
    
    
myfilters = {'SITE':'VELIZY','MEDIA':'Email'}
df_filter = filter_df(df, myfilters)


In [35]:
df_filter

,SITE,CLIENT,FLUX_ACTIVITE,SECTEUR_ACTIVITE,SERVICE_ACTIVITE,MEDIA,TYPEDOSSIER,REGROUPEMENTTDB,TOTAL
DATEDATA,,,,,,,,,
2019-02-06,VELIZY,KLESIA - MUTUELLE GEODIS,Flux BAE-BIA,Contrats- Cotisations,CONTRAT - COTISATION,Email,Adhésion - Portabilité,Dossiers Contractuels,3
2019-09-30,VELIZY,KLESIA PREVOYANCE,Flux Prestation,Prestations,PRESTATION,Email,Factures Optiques,Dossiers Prestations,2
2019-03-27,VELIZY,K2 - MUTUELLE DE NATIXIS,Flux Administratif,Contrats- Cotisations,CONTRAT - COTISATION,Email,Modification fichiers,Dossiers Administratifs,1
2019-02-28,VELIZY,KLESIA - CARCEPT PREVOYANCE,Flux Administratif,Contrats- Cotisations,CONTRAT - COTISATION,Email,Modification de contrat entreprise,Dossiers Administratifs,2
2018-09-11,VELIZY,K2 - MUTUELLE FAMILIALE DES CHEMINOTS DE FRANCE,Flux E/S,Contrats- Cotisations,CONTRAT - COTISATION,Email,Décès,Dossiers Prévoyance,1
...,...,...,...,...,...,...,...,...,...
2019-06-27,VELIZY,K2 - MUTUELLE FAMILIALE DES CHEMINOTS DE FRANCE,Flux Prestation,Prestations,PRESTATION,Email,Prestations,Dossiers Prestations,38
2019-08-30,VELIZY,K2 - MUTUELLE DE PONTOISE,Flux Devis,Prestations,PRESTATION,Email,Devis dentaire,Dossiers Devis,1
2019-10-24,VELIZY,KLESIA - CARCEPT PREVOYANCE,Flux Administratif,Contrats- Cotisations,CONTRAT - COTISATION,Email,Attestation SS,Dossiers EDI,3


### Create Aggregated Df with DatTime Index

In [37]:
df2 = df_filter.groupby('DATEDATA').sum()
df2 = df2.asfreq(freq='B')

In [39]:
df2.index

DatetimeIndex(['2018-08-01', '2018-08-02', '2018-08-03', '2018-08-06',
               '2018-08-07', '2018-08-08', '2018-08-09', '2018-08-10',
               '2018-08-13', '2018-08-14',
               ...
               '2020-06-10', '2020-06-11', '2020-06-12', '2020-06-15',
               '2020-06-16', '2020-06-17', '2020-06-18', '2020-06-19',
               '2020-06-22', '2020-06-23'],
              dtype='datetime64[ns]', name='DATEDATA', length=495, freq='B')

### Split in train test to compare with predictions

In [ ]:
from scipy.stats import boxcox

df3, l = boxcox(df2['TOTAL'])

In [ ]:
l

In [ ]:
df2['TOTAL']=df3

In [ ]:
train = df2[df2.index<'2019-10-01']
test = df2[df2.index>='2019-10-01']

In [ ]:
plt.rcParams['figure.figsize'] = [8, 5]
train.plot();

In [ ]:
train.info()

### Decompose

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
res = seasonal_decompose(train.TOTAL.interpolate(),period=5, model='additive')

# resplot = res.plot()

fig, (ax0,ax1,ax2,ax3) = plt.subplots(4,1, figsize=(20,8))
ax0.set_ylabel('TOTobAL')
ax1.set_ylabel('TREND')
ax2.set_ylabel('SEASONAL')
ax3.set_ylabel('RESIDUALS')


res.observed.plot(ax=ax0);
res.trend.plot(ax=ax1);
res.seasonal.plot(ax=ax2);
res.resid.plot(ax=ax3);

In [ ]:
res.observed[:10]

In [ ]:
plt.plot(res.observed[:10])

In [ ]:
plt.plot(train.TOTAL[:10])

### ACF / PACF

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plt.rcParams['figure.figsize'] = [8, 5]
plot_acf(res.observed, zero=False);
plot_pacf(res.observed, zero=False);

Shows a linear trend in ACF --> differentiate

In [ ]:
traindiff = train.diff(periods=5)[5:]

In [ ]:
res = seasonal_decompose(traindiff.TOTAL.interpolate(),period=5, model='additive')

In [ ]:
plt.rcParams['figure.figsize'] = [8, 5]
plot_acf(res.observed,zero=False);
plot_pacf(res.observed,zero=False);

order one diff seems ok

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

### Create SARIMA model

In [ ]:
from statsmodels.tsa.x13 import x13_arima_select_order

In [ ]:
myorder = (2,1,2)
myorder_season = (0,1,1,5)

In [ ]:
model = SARIMAX(train, order=myorder, seasonal_order=myorder_season)
modfit = model.fit()

In [ ]:
plt.rcParams['figure.figsize'] = [8, 5]
plt.plot(modfit.resid);
plot_acf(modfit.resid,zero=False);
plot_pacf(modfit.resid,zero=False);
# plt.hist(modfit.resid)

In [ ]:

plt.rcParams['figure.figsize'] = [15, 10]
modfit.plot_diagnostics();


In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox

acorr_ljungbox(modfit.resid,lags=None,period=5,return_df=True, boxpierce=True )

In [ ]:
modfit.resid

In [ ]:
# RMSE
import math
math.sqrt(modfit.sse)

In [ ]:
pred = modfit.predict('2019-10-01','2019-12-31')
pred

In [ ]:
graph = train.copy()
test=test.rename(columns={'TOTAL':'TEST'})
graph = graph.append(test)
graph['pred'] = pred

In [ ]:
graph

In [ ]:
# Zoom
graphend = graph[graph.index>'2019-05-01']

In [ ]:
fig = plt.figure(figsize=(15,10))
plt.plot(graphend);

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

mod = ExponentialSmoothing(train,trend="add",damped=True).fit(use_boxcox=True, remove_bias=True)

In [ ]:
mod.params

In [ ]:
mod.forecast(22)